In [1]:
require 'nn'
require 'optim'

In [2]:
require 'csvigo'

true	


In [3]:
loaded = csvigo.load('logistic-regression/example-logistic-regression.csv')

<csv>	parsing file: logistic-regression/example-logistic-regression.csv	


<csv>	tidying up entries	


<csv>	returning tidy table	


In [4]:
-- first convert each variable list to a tensor:
brands = torch.Tensor(loaded.brand)
females = torch.Tensor(loaded.female)
ages = torch.Tensor(loaded.age)

In [5]:
-- copy all the input variables into a single tensor:
dataset_inputs = torch.Tensor( (#brands)[1],2 )
dataset_inputs[{ {},1 }] = females
dataset_inputs[{ {},2 }] = ages

In [6]:
dataset_outputs = brands

In [7]:
numberOfBrands = torch.max(dataset_outputs) - torch.min(dataset_outputs) + 1

In [8]:
-- summarize the data
function summarizeData()
   function p(name,value) 
      print(string.format('%20s %f', name, value) )
   end
   p('number of brands', numberOfBrands)
   p('min brand', torch.min(brands))
   p('max brand', torch.max(brands))
   
   p('min female', torch.min(females))
   p('max female', torch.max(females))
   
   p('min age', torch.min(ages))
   p('max age', torch.max(ages))
end
summarizeData()

    number of brands 3.000000	
           min brand 1.000000	
           max brand 3.000000	
          min female 0.000000	
          max female 1.000000	
             min age 24.000000	
             max age 38.000000	


In [9]:
linLayer = nn.Linear(2,3)

In [10]:
softMaxLayer = nn.LogSoftMax()  -- the input and output are a single tensor

In [11]:
model = nn.Sequential()
model:add(linLayer)
model:add(softMaxLayer)


In [12]:
criterion = nn.ClassNLLCriterion()

In [13]:
x, dl_dx = model:getParameters()

In [14]:
feval = function(x_new)
   -- set x to x_new, if differnt
   -- (in this simple example, x_new will typically always point to x,
   -- so the copy is really useless)
   if x ~= x_new then
      x:copy(x_new)
   end

   -- select a new training sample
   _nidx_ = (_nidx_ or 0) + 1
   if _nidx_ > (#dataset_inputs)[1] then _nidx_ = 1 end

   local inputs = dataset_inputs[_nidx_]
   local target = dataset_outputs[_nidx_]

   -- reset gradients (gradients are always accumulated, to accomodate 
   -- batch methods)
   dl_dx:zero()

   -- evaluate the loss function and its derivative wrt x, for that sample
   local loss_x = criterion:forward(model:forward(inputs), target)
   model:backward(inputs, criterion:backward(model.output, target))

   -- return loss(x) and dloss/dx
   return loss_x, dl_dx
end

In [15]:
?nn

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
Overview


Each module of a network is composed of Modules and there
are several sub-classes of Module available: container classes like
Sequential, Parallel and
Concat , which can contain simple layers like
Linear, Mean, Max and
Reshape, as well as convolutional layers, and transfer
functions like Tanh.

Loss functions are implemented as sub-classes of
Criterion. They are helpful to train neural network on
classical tasks.  Common criterions are the Mean Squared Error
criterion implemented in MSECriterion and the
cross-entropy criterion implemented in
ClassNLLCriterion.

Finally, the StochasticGradient class provides a
high level way to train the neural network of choice, even though it is
easy with a simple for loop to train a neural network yourself.	
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	



In [16]:
sgd_params = {
   learningRate = 1e-3,
   learningRateDecay = 1e-4,
   weightDecay = 0,
   momentum = 0
}

In [17]:
epochs = 1e2

In [18]:
for i = 1,epochs do

   -- this variable is used to estimate the average loss
   current_loss = 0

   -- an epoch is a full loop over our training data
   for i = 1,(#dataset_inputs)[1] do

      -- optim contains several optimization algorithms. 
      -- All of these algorithms assume the same parameters:
      --   + a closure that computes the loss, and its gradient wrt to x, 
      --     given a point x
      --   + a point x
      --   + some parameters, which are algorithm-specific

      _,fs = optim.sgd(feval,x,sgd_params)

      -- Functions in optim all return two things:
      --   + the new x, found by the optimization method (here SGD)
      --   + the value of the loss functions at all points that were used by
      --     the algorithm. SGD only estimates the function once, so
      --     that list just contains one value.

      current_loss = current_loss + fs[1]
   end

   -- report average error on epoch
   current_loss = current_loss / (#dataset_inputs)[1]
   print('epoch = ' .. i .. 
	 ' of ' .. epochs .. 
	 ' current loss = ' .. current_loss)

end

epoch = 1 of 100 current loss = 1.4175289120598	


epoch = 2 of 100 current loss = 1.1750891574662	


epoch = 3 of 100 current loss = 1.1596910515246	


epoch = 4 of 100 current loss = 1.1464358987213	


epoch = 5 of 100 current loss = 1.1349194006369	


...

epoch = 96 of 100 current loss = 1.0116084635622	


epoch = 97 of 100 current loss = 1.0115967530372	


epoch = 98 of 100 current loss = 1.0115888002392	


epoch = 99 of 100 current loss = 1.0115844881925	


epoch = 100 of 100 current loss = 1.0115837041716	


In [19]:
-- return index of largest value
function maxIndex(a,b,c)
   if a >=b and a >= c then return 1 
   elseif b >= a and b >= c then return 2
   else return 3 end
end

In [20]:
function predictText(age, female)
   --   1: calculate the "logit's"
   --      The coefficients come from the text.
   --      If you download the R script and run it, you may see slightly
   --      different results.
   local logit1 = 0
   local logit2 = -11.774655 + 0.523814 * female + 0.368206 * age
   local logit3 = -22.721396 + 0.465941 * female + 0.685908 * age

   --   2: calculate the unnormalized probabilities
   local uprob1 = math.exp(logit1)
   local uprob2 = math.exp(logit2)
   local uprob3 = math.exp(logit3)

   --   3: normalize the probabilities
   local z = uprob1 + uprob2 + uprob3
   local prob1 = (1/z) * uprob1
   local prob2 = (1/z) * uprob2
   local prob3 = (1/z) * uprob3

   return maxIndex(prob1, prob2, prob3), prob1, prob2, prob3
end

-- return predicted brand and the probabilities of each brand
-- for our model
function predictOur(age, female)
   local input = torch.Tensor(2)
   input[1] = female  -- must be in same order as when the model was trained!
   input[2] = age
   local logProbs = model:forward(input)  
   --print('predictOur', age, female, input)
   local probs = torch.exp(logProbs)
   --print('logProbs', logProbs)
   --print('probs', probs[1], probs[2], probs[3] )
   local prob1, prob2, prob3 = probs[1], probs[2], probs[3]
   return maxIndex(prob1, prob2, prob3), prob1, prob2, prob3
end
      
counts = {}

function makeKey(age, brand, female)
   -- return a string containing the values

   -- Note that returning a table will not work, because each
   -- table is unique.

   -- Because Lua interns the strings, a string with a given sequence
   -- of characters is stored only once.
   return string.format('%2d%1d%1f', age, brand, female)
end

for i = 1,(#brands)[1] do
   local brand = brands[i]
   local female = females[i]
   local age = ages[i]
   local key = makeKey (age, brand, female)
   counts[key] = (counts[key] or 0) + 1
end

-- return probability of each brand conditioned on age and female
function actualProbabilities(age, female)
   function countOf(age, brand, female)
      return counts[makeKey(age, brand, female)] or 0
   end
   local count1 = countOf(age, 1, female)
   local count2 = countOf(age, 2, female)
   local count3 = countOf(age, 3, female)
   local sumCounts = count1 + count2 + count3
   if sumCounts == 0 then
      return 0, 0, 0
   else
      return count1/sumCounts, count2/sumCounts, count3/sumCounts
   end
end


print(' ')
print('summary of data')
summarizeData()

print(' ')
print('training variables')
for k,v in pairs(sgd_params) do
   print(string.format('%20s %f', k, v))
end
print(string.format('%20s %f', 'epochs', epochs))

print(' ')
print('current loss', current_loss)

-- print the headers 
print(' ')
lineFormat = '%-6s %-3s| %-17s | %-17s | %-17s | %-1s %-1s %-1s'
print(
   string.format(lineFormat,
		 '', '', 
		 'actual probs', 'text probs', 'our probs', 
		 'best', '', ''))
choices = 'brnd1 brnd2 brnd3'
print(string.format(lineFormat,
		    'female', 'age', 
		    choices, choices, choices, 
		    'a', 't', 'o'))

-- print each row in the table

function formatFemale(female)
   return string.format('%1d', female)
end

function formatAge(age)
   return string.format('%2d', age)
end

function formatProbs(p1, p2, p3)
   return string.format('%5.3f %5.3f %5.3f', p1, p2, p3)
end

function indexString(p1, p2, p3)
   -- return index of highest probability or '-' if nearly all zeroes
   if p1 < 0.001 and p2 < 0.001 and p3 < 0.001 then
      return '-'
   else 
      return string.format('%1d', maxIndex(p1, p2, p3))
   end
end

-- print table rows and accumulate accuracy
for female = 0,1 do
   for age = torch.min(ages),torch.max(ages) do
      -- calculate the actual probabilities in the training data
      local actual1, actual2, actual3 = actualProbabilities(age, female)
      -- calculate the prediction and probabilities using the model in the text
      local textBrand, textProb1, textProb2, textProb3 = 
	 predictText(age, female)
      -- calculate the probabilities using the model we just trained
      --print("main", age, female)
      local ourBrand, ourProb1, ourProb2, ourProb3 = 
	 predictOur(age, female)
      print(
	 string.format(lineFormat,
		       formatFemale(female), 
		       formatAge(age),
		       formatProbs(actual1, actual2, actual3),
		       formatProbs(textProb1, textProb2, textProb3),
		       formatProbs(ourProb1, ourProb2, ourProb3),
		       indexString(actual1,actual2,actual3),
		       indexString(textProb1,textProb2,textProb3),
		       indexString(ourProb1,ourProb2,ourProb3))
	   )
   end
end

 	
summary of data	
    number of brands 3.000000	
           min brand 1.000000	
           max brand 3.000000	
          min female 0.000000	
          max female 1.000000	
             min age 24.000000	


             max age 38.000000	
 	
training variables	
         evalCounter 73500.000000	


   learningRateDecay 0.000100	
        learningRate 0.001000	
         weightDecay 0.000000	
            momentum 0.000000	
              epochs 100.000000	
 	
current loss	1.0115837041716	
 	
          | actual probs      | text probs        | our probs         | best    	
female age| brnd1 brnd2 brnd3 | brnd1 brnd2 brnd3 | brnd1 brnd2 brnd3 | a t o	


0      24 | 1.000 0.000 0.000 | 0.948 0.050 0.002 | 0.097 0.198 0.705 | 1 1 3	
0      25 | 0.000 0.000 0.000 | 0.926 0.071 0.004 | 0.094 0.198 0.708 | - 1 3	


0      26 | 1.000 0.000 0.000 | 0.894 0.099 0.007 | 0.091 0.197 0.712 | 1 1 3	
0      27 | 1.000 0.000 0.000 | 0.851 0.136 0.013 | 0.088 0.197 0.715 | 1 1 3	


...

1      34 | 0.150 0.400 0.450 | 0.134 0.477 0.389 | 0.051 0.183 0.766 | 3 2 3	


1      35 | 0.042 0.250 0.708 | 0.084 0.432 0.484 | 0.049 0.182 0.768 | 3 3 3	


1      36 | 0.109 0.345 0.545 | 0.050 0.374 0.576 | 0.048 0.181 0.771 | 3 3 3	


1      37 | 0.000 0.294 0.706 | 0.029 0.311 0.660 | 0.046 0.181 0.773 | 3 3 3	


1      38 | 0.071 0.214 0.714 | 0.016 0.252 0.732 | 0.044 0.180 0.776 | 3 3 3	
